In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "canina_cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.005, 1),
    "--max_iter": choice(100, 300, 500, 700, 1000)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

#### Copy train.py script into "./training" to use it in ScriptRunConfig object.
import shutil

shutil.copy('train.py', "./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                     script='train.py',
                     arguments=['--C', '--max_iter'],
                     compute_target=compute_target,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrive-model',
                                   model_path = 'outputs/model.pkl')

In [ ]:
# This print is to see the best model
#print(best_hyperdrive_model.name, best_hyperdrive_model.id, best_hyperdrive_model.version, sep='\t')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path_data, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')
    

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "primary_metric" : 'r2_score',
    "featurization" : 'auto',
    "verbosity" : logging.INFO,
    "n_cross_validations" : 5    
}

# Split data into train and test sets using parameter test_size.

automl_config = AutoMLConfig(
    task='regression',
    debug_log='automl_errors.log',
    compute_target=compute_target,
    training_data=x,
    label_column_name=y,
    test_size=0.2,    
    **automl_settings)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

automl_run.wait_for_completion(show_output=False)

best_run_automl, best_automl_model = automl_run.get_output()

automl_model_name = best_run_automl.properties["model_name"]
registered_model = automl_run.register_model(model_name=automl_model_name,
                                                 description = "Automl model",
                                                 tags=None)


In [ ]:
test_run_automl = next(best_run_automl.get_children(type='automl.model_test'))
test_run_automl.wait_for_completion(show_output=False, wait_post_processing=True)

In [ ]:
# Test the performance of the automl model.
test_run_metrics = test_run_automl.get_metrics()
for m, val in test_run_metrics.items():
    print(f"{m}: {val}")

In [ ]:
# Remove the compute cluster following training.
compute_target.delete()